IMPORT
--

In [1]:
from google.colab.patches import cv2_imshow
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from google.colab import files
uplot = files.upload()

Reading

In [33]:
citra = cv.imread('strawberry.png')

In [34]:
gray = cv.cvtColor(citra, cv.COLOR_BGR2GRAY)

Displaying

In [ ]:
from google.colab.patches import cv2_imshow
cv2_imshow(citra)

Noise Gaussian

In [ ]:
import cv2
import scipy
import scipy.stats
import random

def NoiseGaussian(image,sigma):
    # Gaussian noise
    img = image.astype(np.int16)
    mu = 0
    for i in range(img.shape[0]):
        for j in range(img.shape[1]):
            for k in range(img.shape[2]):
                img[i, j, k] = img[i, j, k] + random.gauss(mu=mu, sigma=sigma)
    img[img > 255] = 255
    img[img < 0] = 0
    img = img.astype(np.uint8)
    return img

Gauss= NoiseGaussian(citra,18)
cv2_imshow(Gauss)


Pepper

In [ ]:
def spNoisy(image,s_vs_p = 0.5,amount = 0.004):
    out = np.copy(image)
    num_salt = np.ceil(amount * image.size * s_vs_p)
    coords = [np.random.randint(0, i - 1, int(num_salt)) for i in image.shape]
    out[tuple(coords)] = 1
    num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
    coords = [np.random.randint(0, i - 1, int(num_pepper)) for i in image.shape]
    out[tuple(coords)] = 0
    return out

Pepper = spNoisy(citra)
cv2_imshow(Pepper)

Arithmetic mean filtering

In [ ]:
def ArithmeticMean(image):
    new_image = np.zeros(image.shape)
    image = cv2.copyMakeBorder(image,1,1,1,1,cv2.BORDER_DEFAULT)
    for i in range(1,image.shape[0]-1):
        for j in range(1,image.shape[1]-1):
            new_image[i-1,j-1] = np.mean(image[i-1:i+2,j-1:j+2])
    new_image = (new_image-np.min(image))*(255/np.max(image))
    return new_image.astype(np.uint8)

def rgbArithmeticMean(image):
    r,g,b = cv2.split(image)
    r = ArithmeticMean(r)
    g = ArithmeticMean(g)
    b = ArithmeticMean(b)
    return cv2.merge([r,g,b])

Gauss= NoiseGaussian(citra,18)
GaussArith = rgbArithmeticMean(Gauss)

Pepper = spNoisy(citra)
PepperArith =rgbArithmeticMean(Pepper)

result = np.hstack((Gauss, GaussArith, Pepper, PepperArith))
cv2_imshow(result)

Geometric mean filtering

In [ ]:
def GeometricMeanOperator(roi):
    roi = roi.astype(np.float64)
    p = np.prod(roi)
    return p ** (1 / (roi.shape[0] * roi.shape[1]))

def GeometricMean(image):
    new_image = np.zeros(image.shape)
    image = cv2.copyMakeBorder(image, 1, 1, 1, 1, cv2.BORDER_DEFAULT)
    for i in range(1, image.shape[0] - 1):
        for j in range(1, image.shape[1] - 1):
            new_image[i - 1, j - 1] = GeometricMeanOperator(image[i - 1:i + 2, j - 1:j + 2])
    new_image = (new_image - np.min(image)) * (255 / np.max(image))
    return new_image.astype(np.uint8)

def rgbGemotriccMean(image):
    r,g,b = cv2.split(image)
    r = GeometricMean(r)
    g = GeometricMean(g)
    b = GeometricMean(b)
    return cv2.merge([r,g,b])

Gauss= NoiseGaussian(citra,18)
GaussGeo = rgbGemotriccMean(Gauss)

PepperGeo = spNoisy(citra)
rgbGemotriccMean(Pepper)

result = np.hstack((Gauss, GaussGeo, Pepper, PepperGeo))
cv2_imshow(result)

Harmonic mean filtering

In [ ]:
def HarmonicMeanOperator(roi):
    roi = roi.astype(np.float64)
    if 0 in roi:
        roi = 0
    else:
        roi = scipy.stats.hmean(roi.reshape(-1))
    return roi
def HarmonicMean(image):
    new_image = np.zeros(image.shape)
    image = cv2.copyMakeBorder(image,1,1,1,1,cv2.BORDER_DEFAULT)
    for i in range(1,image.shape[0]-1):
        for j in range(1,image.shape[1]-1):
            new_image[i-1,j-1] =HarmonicMeanOperator(image[i-1:i+2,j-1:j+2])
    new_image = (new_image-np.min(image))*(255/np.max(image))
    return new_image.astype(np.uint8)
def rgbHarmonicMean(image):
    r,g,b = cv2.split(image)
    r = HarmonicMean(r)
    g = HarmonicMean(g)
    b = HarmonicMean(b)
    return cv2.merge([r,g,b])

Gauss= NoiseGaussian(citra,18)
GaussHarmonic = rgbHarmonicMean(Gauss)

Pepper = spNoisy(citra)
PepperHarmonic = rgbHarmonicMean(Pepper)

result = np.hstack((Gauss, GaussHarmonic, Pepper, PepperHarmonic))
cv2_imshow(result)

Contraharmonic mean filtering

In [ ]:
def ContraharmonicMeanOperator(roi,q):
    roi = roi.astype(np.float64)
    return np.mean((roi)**(q+1))/np.mean((roi)**(q))
def ContraharmonicMean(image,q):
    new_image = np.zeros(image.shape)
    image = cv2.copyMakeBorder(image,1,1,1,1,cv2.BORDER_DEFAULT)
    for i in range(1,image.shape[0]-1):
        for j in range(1,image.shape[1]-1):
            new_image[i-1,j-1] = ContraharmonicMeanOperator(image[i-1:i+2,j-1:j+2],q)
    new_image = (new_image-np.min(image))*(255/np.max(image))
    return new_image.astype(np.uint8)
def rgbContraharmonicMean(image,q):
    r,g,b = cv2.split(image)
    r = ContraharmonicMean(r,q)
    g = ContraharmonicMean(g,q)
    b = ContraharmonicMean(b,q)
    return cv2.merge([r,g,b])

Gauss= NoiseGaussian(citra,18)
GaussCHarmonic = rgbContraharmonicMean(Gauss,2)

Pepper = spNoisy(citra)
PepperCHarmonic = rgbContraharmonicMean(Pepper,2)

result = np.hstack((Gauss, GaussCHarmonic, Pepper, PepperCHarmonic))
cv2_imshow(result)

In [ ]:
plt.subplot(251),plt.imshow(cv2.cvtColor(Gauss, cv2.COLOR_BGR2RGB)),plt.axis('off')
plt.subplot(252),plt.imshow(cv2.cvtColor(Pepper, cv2.COLOR_BGR2RGB)),plt.axis('off')
plt.subplot(253),plt.imshow(cv2.cvtColor(GaussArith, cv2.COLOR_BGR2RGB)),plt.axis('off')
plt.subplot(254),plt.imshow(cv2.cvtColor(PepperArith, cv2.COLOR_BGR2RGB)),plt.axis('off')
plt.subplot(255),plt.imshow(cv2.cvtColor(GaussGeo, cv2.COLOR_BGR2RGB)),plt.axis('off')
plt.subplot(256),plt.imshow(cv2.cvtColor(PepperGeo, cv2.COLOR_BGR2RGB)),plt.axis('off')
plt.subplot(257),plt.imshow(cv2.cvtColor(GaussHarmonic, cv2.COLOR_BGR2RGB)),plt.axis('off')
plt.subplot(258),plt.imshow(cv2.cvtColor(PepperHarmonic, cv2.COLOR_BGR2RGB)),plt.axis('off')
plt.subplot(259),plt.imshow(cv2.cvtColor(GaussCHarmonic, cv2.COLOR_BGR2RGB)),plt.axis('off')
plt.subplot(2,5,10),plt.imshow(cv2.cvtColor(PepperCHarmonic, cv2.COLOR_BGR2RGB)),plt.axis('off')
plt.show()